In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import utils

# import os
# import openai
# openai.api_key = os.environ["api key"]
import os
# Sahi tarika API key set karne ka:
os.environ["GROQ_API_KEY"] = "api key"

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PyMuPDFReader

# Explicitly specify PyMuPDFReader for .pdf files
documents = SimpleDirectoryReader(
    input_files=[r'C:\Users\DELL\OneDrive\Desktop\Sarfraz\PW Data Science\Project\RAG_Application_Using_LLM\Medical_Cost_Prediction.pdf'],
    file_extractor={".pdf": PyMuPDFReader()}
).load_data()

print(f"Number of documents loaded: {len(documents)}")

Number of documents loaded: 4


In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

4 

<class 'llama_index.core.schema.Document'>
Doc ID: 972f00fc-a011-4063-968e-6fde18085b49
Text: 11/28/23, 11:00 PM README.md - Grip localhost:6419 1/4 README.md
Medical Cost Prediction Predicting medical costs of individuals based
on different features using several ML (Regression) algorithms. The
application was deployed on AWS EC2 through AWS ECR (Dockerized
Container). Dataset The Medical Cost Prediction consists of around
1300 records ...


In [5]:
# Purana (v0.9): from llama_index import Document
# Naya (v0.10+):
from llama_index.core import Document

# Documents ko merge karo
document = Document(text='\n\n'.join(doc.text for doc in documents))

## Auto-Merging Retrieval Setup

In [6]:
from utils import build_automerging_index, get_automerging_query_engine

# Naya v0.10+ import path:
from llama_index.llms.openai import OpenAI

# Agar aap Groq use karna chahte hain (jo humne pehle test kiya tha):
# from llama_index.llms.groq import Groq
# llm = Groq(model="llama-3.3-70b-versatile")

index = build_automerging_index(
    [document],
    llm=OpenAI(model='gpt-3.5-turbo', temperature=0.1),
    save_dir='merging_index'
)

2026-01-16 18:44:04,459 - INFO - setup plugin alembic.autogenerate.schemas
2026-01-16 18:44:04,461 - INFO - setup plugin alembic.autogenerate.tables
2026-01-16 18:44:04,461 - INFO - setup plugin alembic.autogenerate.types
2026-01-16 18:44:04,461 - INFO - setup plugin alembic.autogenerate.constraints
2026-01-16 18:44:04,461 - INFO - setup plugin alembic.autogenerate.defaults
2026-01-16 18:44:04,461 - INFO - setup plugin alembic.autogenerate.comments
2026-01-16 18:44:20,421 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 92ab37f5-523c-4e36-a43b-8ef9139f57c9)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/./README.md
2026-01-16 18:44:32,270 - WARNING - '(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 92ab37f5-523c-4e36-a43b-8ef

In [ ]:
# Query engine ban gaya hai
query_engine = get_automerging_query_engine(index, similarity_top_k=10)

# Ab apna sawaal puchein
response = query_engine.query("What are the key points of the medical cost prediction?")
print(str(response))

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

## TruLens Evaluation

In [ ]:
from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Two Layers

In [ ]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

In [ ]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [ ]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
questions = []
with open('eval_questions.txt','r') as file:
    for line in file:
        item = line.strip()
        questions.append(item)

In [ ]:
questions

['What are the technologies used in the project?',
 'What is the installation process?',
 'Can you talk about the dataset used in the project?',
 'Can you explain the usage of the project?',
 'What is the process of deployment used in the project?']

In [ ]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [ ]:
run_evals(questions, tru_recorder, auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - G

In [ ]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.35625,1.0,0.233333,5.8,0.001836


In [ ]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


python(3878) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.34:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Three Layers


In [ ]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512,128],
)

In [ ]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [ ]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
run_evals(questions, tru_recorder, auto_merging_engine_1)

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...



python(3891) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - G

In [ ]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.40000,1.0,0.596667,5.8,0.001836
app_1,0.36875,1.0,0.666667,5.8,0.001837


In [ ]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.4.34:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>